# This notebook will aim to use the trained models weights and load it on the inference model to generate predictions and novel sequences

In [45]:
from models.arcs import load_inf_model, GenPhiloText
from utilities.loaders import load_file
from utilities.preprocessors import preprocess, get_chars, map_value_to_index, decode_predictions
import tensorflow as tf

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load raw data

In [46]:
corpus = load_file('./data/notes.txt')
corpus = preprocess(corpus)
chars = get_chars(corpus)
char_to_idx = map_value_to_index(chars)
idx_to_char = map_value_to_index(chars, inverted=True)

In [47]:
char_to_idx.get_vocabulary()

['[UNK]',
 '\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '´',
 'ç',
 'é',
 'ï',
 '—',
 '…']

In [48]:
n_unique = len(char_to_idx.get_vocabulary())

# Declare same hyper params used in training model

In [49]:
emb_dim = 32
n_a = 64
T_x = 100
dense_layers_dims = [64, 32, n_unique] 
batch_size = 128
alpha = 1e-3
lambda_ = 0.8
drop_prob = 0.4
n_epochs =  100

# Declare sample input to build inference model and access .summary()

In [50]:
sample_input = tf.random.uniform(shape=(1, T_x), minval=0, maxval=n_unique - 1, dtype=tf.int32)
sample_h = tf.zeros(shape=(1, n_a))
sample_c = tf.zeros(shape=(1, n_a))

# Redeclare architecture by passing the same hyper params used in training and then load weights

In [51]:
saved_model = GenPhiloText(emb_dim=emb_dim, n_a=n_a, n_unique=n_unique, T_x=T_x, dense_layers_dims=dense_layers_dims)
saved_model([sample_input, sample_h, sample_c])

[<tf.Tensor: shape=(1, 57), dtype=float32, numpy=
 array([[ 1.7750724e-03, -3.9843426e-04, -1.0051613e-03, -8.0314721e-04,
          3.0034729e-03, -7.2323615e-05,  7.2772778e-04,  4.3589112e-04,
         -6.4147147e-04,  2.9683884e-03, -1.5742945e-03,  1.3282284e-03,
         -2.4821979e-04, -1.2024168e-03, -1.7930442e-04, -6.1119220e-04,
         -3.7751309e-04,  4.6796538e-04, -2.0108274e-03,  2.5303004e-04,
          1.1149838e-03,  7.6347159e-04,  1.1012034e-03,  1.3472301e-04,
          9.0014149e-04,  2.4215139e-03, -4.0225673e-04, -8.8095572e-04,
          1.1154832e-03, -2.4556778e-03,  8.1808190e-04, -3.4688317e-04,
         -6.6014729e-04,  1.5380046e-03,  1.4003715e-03,  1.7270783e-04,
         -8.0836238e-04, -4.3934837e-05,  1.4985469e-03, -3.4818565e-04,
         -2.2305813e-04,  1.8582955e-03,  9.9969585e-04, -7.8299039e-05,
          1.6634889e-03, -1.7830601e-04,  2.3238547e-03, -1.2141047e-03,
         -6.7083706e-04,  1.3990734e-03,  1.0971780e-03,  2.2037979e-03,
 

In [52]:
saved_model.summary()

Model: "gen_philo_text_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 char-emb-layer (Embedding)  multiple                  1824      
                                                                 
 lstm-cell (LSTM)            multiple                  24832     
                                                                 
 dense-layer-0 (Dense)       multiple                  4160      
                                                                 
 dense-layer-1 (Dense)       multiple                  2080      
                                                                 
 dense-layer-2 (Dense)       multiple                  1881      
                                                                 
 norm-layer-0 (BatchNormali  multiple                  256       
 zation)                                                         
                                                  

In [53]:
saved_model.load_weights(filepath='./saved/weights/notes_gen_philo_text_95_302.9099.h5')

# Extract trainable layers

In [54]:
layers = saved_model.layers
for layer in layers:
    print(layer.name)

char-emb-layer
lstm-cell
dense-layer-0
dense-layer-1
dense-layer-2
norm-layer-0
norm-layer-1
act-layer-0
act-layer-1
drop-layer-0
drop-layer-1
reshape-layer


In [55]:
char_emb_layer = saved_model.get_layer('char-emb-layer')
lstm_cell = saved_model.get_layer('lstm-cell')
dense_layers = []
norm_layers = []
for layer in layers:
    if "dense" in layer.name:
        dense_layers.append(saved_model.get_layer(layer.name))
    if "norm" in layer.name:
        norm_layers.append(saved_model.get_layer(layer.name))

In [56]:
dense_layers[0].get_weights()

[array([[-1.5397636e-15, -1.6668247e-03,  1.4865319e-03, ...,
          1.3182290e-03,  1.6925022e-03,  1.0884514e-03],
        [ 9.7578686e-22, -9.1989827e-04,  5.1410031e-04, ...,
          7.4651063e-04,  6.0411548e-04, -3.7277990e-04],
        [ 5.4481627e-22, -3.3333926e-03,  3.4297095e-03, ...,
          3.2766573e-03,  3.4724199e-03,  2.5732655e-03],
        ...,
        [-5.5700574e-16, -2.8473698e-03,  4.9765375e-03, ...,
          3.8931344e-03,  4.4233333e-03,  5.8457926e-03],
        [ 2.8039348e-20,  3.5393266e-03, -2.3225276e-03, ...,
         -3.2931708e-03, -2.2087635e-03, -5.6925585e-04],
        [ 3.3345108e-22, -5.7764421e-03,  3.7177056e-03, ...,
          3.9774869e-03,  3.4261188e-03,  9.5337420e-04]], dtype=float32),
 array([ 0.01083319, -0.00501028,  0.00518756, -0.00128027,  0.00583724,
        -0.00331641, -0.0054764 , -0.00614964,  0.00421741,  0.01883411,
        -0.00266028, -0.00269486, -0.00046412,  0.005468  ,  0.00864111,
         0.00801458,  0.0029481

In [57]:
dense_layers[1].get_weights()

[array([[ 6.2350690e-20, -6.2310029e-08,  3.7449277e-21, ...,
         -4.9313342e-17, -3.2880255e-20, -2.7782339e-19],
        [ 3.9493591e-03, -2.1733889e-02,  1.6101990e-03, ...,
          1.9732858e-03,  1.7414147e-02,  6.9244934e-04],
        [-1.1605115e-03,  6.5325825e-03, -2.4851179e-03, ...,
         -4.4733859e-04, -1.6623579e-02, -4.8930454e-03],
        ...,
        [-1.8718115e-03,  1.0895255e-02, -2.1316977e-03, ...,
         -1.6550759e-03, -1.6433273e-02, -2.4392409e-03],
        [ 5.8647949e-04,  1.2011512e-02, -2.7711662e-03, ...,
         -1.2979350e-03, -1.8666349e-02, -7.1629011e-03],
        [-9.1946824e-03, -6.2809966e-04,  1.0635084e-03, ...,
          1.4957330e-03, -6.3888882e-03, -1.0527691e-03]], dtype=float32),
 array([-0.02543983, -0.0063192 , -0.00357558,  0.00014908, -0.00387982,
         0.00050349, -0.01273545, -0.00458925,  0.0036891 ,  0.01338962,
         0.01281952,  0.00034184,  0.01439871, -0.00282337,  0.00141102,
        -0.01946066,  0.0021315

In [58]:
len(char_emb_layer.get_weights())

1

In [59]:
lstm_cell.get_weights()

[array([[ 0.20460863,  0.0052893 , -0.05215346, ..., -0.01613565,
          0.19322333, -0.01479113],
        [-0.01982943,  0.05135198,  0.00418565, ...,  0.1401884 ,
         -0.03097642, -0.01090095],
        [-0.04459645, -0.09727775,  0.06612745, ..., -0.04054819,
         -0.06387071, -0.15369709],
        ...,
        [-0.06652   , -0.05899383,  0.16990899, ..., -0.01274249,
         -0.12033732, -0.10913877],
        [-0.02243979, -0.06018617,  0.03552597, ..., -0.05711776,
         -0.10861681,  0.19668798],
        [ 0.05537038, -0.06307524,  0.06389742, ...,  0.03749431,
         -0.1282892 , -0.03721353]], dtype=float32),
 array([[ 0.04157104, -0.04595341,  0.24019413, ...,  0.07991451,
          0.18064378,  0.16690779],
        [ 0.07832426,  0.19018726,  0.40133384, ...,  0.20638688,
          0.14659484,  0.05754363],
        [ 0.1042229 ,  0.23313536,  0.04130004, ...,  0.12881975,
          0.03064618,  0.05353494],
        ...,
        [-0.06524871, -0.01979281, -0.1

# Build inference model

In [60]:
inference_model = load_inf_model(char_emb_layer, lstm_cell, dense_layers, norm_layers, char_to_idx, T_x=250)

57
64
tf.Tensor(
[[-inf   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.]], shape=(1, 57), dtype=float32)
250
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
1

In [61]:
inference_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 char-emb-layer (Embedding)  (None, 1, 32)                1824      ['input_3[0][0]',             
                                                                     'reshape-layer[0][0]',       
                                                                     'reshape-layer[1][0]',       
                                                                     'reshape-layer[2][0]',       
                                                                     'reshape-layer[3][0]',       
                                                                     'reshape-layer[4][0]', 

In [62]:
pred_ids = inference_model.predict([sample_input, sample_h, sample_c])
pred_ids

1/1 [==============================] - 132s 132s/step


[array([[0.        , 0.01274152, 0.10081399, 0.00271383, 0.00388942,
         0.0027054 , 0.00275645, 0.00306912, 0.00288128, 0.01282268,
         0.00288445, 0.00921785, 0.00291689, 0.00274599, 0.00291238,
         0.00260984, 0.00276398, 0.00273534, 0.00274615, 0.00277197,
         0.00257161, 0.00256735, 0.00282716, 0.00300831, 0.0037314 ,
         0.05368345, 0.01715724, 0.0266667 , 0.03172769, 0.06890227,
         0.02796936, 0.02177686, 0.04380583, 0.05425133, 0.00379209,
         0.00911108, 0.03498932, 0.02432246, 0.0515828 , 0.0541053 ,
         0.01723298, 0.00325598, 0.04044922, 0.04597805, 0.06086593,
         0.0284125 , 0.01607216, 0.02372165, 0.00416879, 0.02191568,
         0.00329971, 0.00273407, 0.00251329, 0.00258173, 0.00262218,
         0.00334341, 0.00358447]], dtype=float32),
 array([[0.        , 0.01286463, 0.09892637, 0.00280385, 0.00399501,
         0.0027962 , 0.00284845, 0.00316955, 0.00297623, 0.01296031,
         0.00297496, 0.00934673, 0.00301352, 0.00283

In [63]:
tf.argmax(pred_ids[0], axis=-1)

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([2], dtype=int64)>

In [64]:
tf.argmax(pred_ids[1], axis=-1)

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([2], dtype=int64)>